<a href="https://colab.research.google.com/github/kumariabha/OS_LAB/blob/master/Prey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
# Algorithm parameters
max_param = 1.0
min_param = 0.1

# Example usage:
dimension = 10
MaxGen = 100
m = 50

def objective_function(x):
    return np.sum(x**2)  # Sphere function as an example


epsilon_min= 1e-5
epsilon_max= 1e-2
p_follow_up= 0.5



def PPA(MaxGen, m, objective_function, move_prey_parameters, move_predator_parameters):
    # Initialize population
    X = np.random.rand(m, dimension)
    MaxGen = MaxGen

    for gen in range(1, MaxGen + 1):
        # Calculate survival value for each solution member
        SV = [objective_function(xi) for xi in X]

        # Identify predator
        predator_index = np.argmax(SV)
        x_predator = X[predator_index]

        # Set of Prey
        prey_indices = list(range(m))
        prey_indices.remove(predator_index)

        # Move prey individuals
        for i in prey_indices:
            X[i] = move_prey(X[i], x_predator, X,objective_function, SV, epsilon_min, epsilon_max, p_follow_up)

        # Move predator
        x_predator = move_predator(x_predator, X, SV, epsilon_min, epsilon_max)

        # Update solution set
        X[predator_index] = x_predator

    # Return the best solution found by the algorithm
    best_index = np.argmin(SV)
    best_solution = X[best_index]

    return best_solution




def move_prey(xi, x_predator, X, objective_function, SV, epsilon_min, epsilon_max, p_follow_up):
    A = [x for x in X if objective_function(x) > objective_function(xi)]

    if A:
        k = len(xi)
        u = np.random.rand(k)
        z = xi + epsilon_min * np.random.rand() * u
        zk_1 = xi
        u_k1 = np.zeros(k)

        SV_z = [objective_function(zj) for zj in np.vstack((z, zk_1))]
        l = np.argmax(SV_z)
        xi = X[l]
    else:
        if np.random.rand() <= p_follow_up:
            yi = calculate_yi(xi,X,SV, epsilon_min, gamma)
            k = len(xi)
            y = np.random.rand(k)
            z = xi + epsilon_min * y
            SV_z = [objective_function(zj) for zj in z]
            l = np.argmax(SV_z)
            yr = y[l]
            xi = update_xi(xi, yi, yr, epsilon_max)
        else:
            yr = np.random.rand(len(xi))
            d1, d2 = calculate_d1_d2(xi, yr, x_predator, objective_function, epsilon_min)
            if d1 >= d2:
                yr = -yr
            xi = xi + epsilon_max * np.random.rand() * yr

    return xi

def calculate_yi(xi, X, SV, epsilon, gamma):
    # Calculate yi using the modified formula (Eq. 3.3)
    dimension = len(xi)
    num_solutions = len(X)

    yi = np.zeros(dimension)

    for j in range(num_solutions):
        rij = np.linalg.norm(X[j] - xi)
        if rij > 0:
            exponential_term = np.exp(gamma * SV[j])  # SV can be replaced with e(SV) as mentioned
            yi += exponential_term * epsilon * rij * (X[j] - xi)

    return yi


def update_xi(xi, yi, yr, epsilon_max):
    # Implementation of Eq. (3.8)
    xi = xi + l_max * epsilon_min * yi / np.linalg.norm(yi) + epsilon_max * yr / np.linalg.norm(yr)
    return xi

def calculate_d1_d2(xi,  yr,x_predator):
    # Calculate distances d1 and d2 (Eqs. 3.4 and 3.5)
    d1 = np.linalg.norm(x_predator - (xi + yr))
    d2 = np.linalg.norm(x_predator - (xi - yr))

    return d1, d2

def choose_random_direction(yr, d1, d2):
    # Choose yr based on distances d1 and d2
    if d1 < d2:
        return yr
    else:
        return -yr  # Take the negative of yr




def move_predator(x_predator, X, SV,epsilon_min, epsilon_max):
    dimension = len(x_predator)

    # Select prey individuals with higher survival value
    prey_indices = [i for i, xi in enumerate(X) if objective_function(xi) >= objective_function(x_predator)]

    if prey_indices:
        # Random direction generation
        yr = np.random.rand(dimension)

        # Update x_predator using Eq. (3.9)
        x_predator = x_predator + (max_param - min_param) * np.random.rand() * yr

    return x_predator


# Algorithm parameters
max_param = 1.0
min_param = 0.1







# Example usage:
dimension = 10
MaxGen = 100
m = 50

def objective_function(x):
    return np.sum(x**2)  # Sphere function as an example

move_prey_parameters = {
    'epsilon_min': 1e-5,
    'epsilon_max': 1e-2,
    'p_follow_up': 0.5
}

move_predator_parameters = {
    'epsilon_min': 1e-5,
    'epsilon_max': 1e-2
}


result = PPA(MaxGen, m, objective_function, move_prey_parameters, move_predator_parameters)
print("Best Solution:", result)
import numpy as np

def move_prey_general(xi, x_predator, best_prey, l_max, epsilon_min, epsilon_max):
    dimension = len(xi)

    # Random directions generation
    yi = np.random.rand(dimension)
    yr = np.random.rand(dimension)

    # Update prey position using Eq. (3.8)
    xi = xi + l_max * epsilon_min * yi / np.linalg.norm(yi) + epsilon_max * yr / np.linalg.norm(yr)

    return xi

def move_prey_best(xi, best_prey, epsilon_min, epsilon_max):
    dimension = len(xi)

    # Random direction generation
    yl = np.random.rand(dimension)

    # Update best prey position using Eq. (3.9)
    xi = xi + epsilon_min * np.random.rand() * yl / np.linalg.norm(yl)

    return xi

# Example usage:
dimension = 10
xi = np.random.rand(dimension)  # Example prey individual
x_predator = np.random.rand(dimension)  # Example predator individual
best_prey = np.random.rand(dimension)  # Example best prey individual

# Algorithm parameters
l_max = 1.0  # Replace with the actual value
epsilon_min = 1e-5  # Replace with the actual value
epsilon_max = 1e-2  # Replace with the actual value

# Move general prey
result_general = move_prey_general(xi, x_predator, best_prey, l_max, epsilon_min, epsilon_max)
print("Updated General Prey:", result_general)

# Move best prey
result_best = move_prey_best(xi, best_prey, epsilon_min, epsilon_max)
print("Updated Best Prey:", result_best)


Best Solution: [0.03365558 0.23566046 0.50327399 0.40164    0.07949059 0.80105376
 0.99616443 0.39106076 0.24305752 0.08491905]
Updated General Prey: [1.00390668 0.68669976 0.84410904 0.81490015 0.78140855 0.30293254
 0.31812601 0.39984075 0.58589461 0.07429767]
Updated Best Prey: [0.99956477 0.68337523 0.84102816 0.810859   0.77741168 0.30230937
 0.31703684 0.39646453 0.58403379 0.07082964]
